#### 이번 대회에서는 범주형 변수 전처리를 위해 One-Hot Encoding과 for문을 사용했습니다.

#### 이는 train data로 fit한 One-Hot Encoder로 test data를 transform할 경우,
#### train data에는 속하지 않은 데이터가 test data에 있을 가능성이 있어 에러가 발생할 수 있기 때문입니다.

#### 이를 방지하기 위해 예외적인 상황에 대처할 수 있는 코드를 삽입해서 이중 for문을 작성했습니다.
#### 참고해 주시길 바랍니다.

# Import

In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

# Fixed RandomSeed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Data Load

In [4]:
train_df = pd.read_csv('./train.csv')

In [ ]:
train_x = train_df.drop(columns=['ID', 'monthlyRent(us_dollar)'])
train_y = train_df['monthlyRent(us_dollar)']

In [ ]:
test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

# Data Pre-processing

#### 범주형 변수는 크게 명목형 변수와 순서형 변수로 나눌 수 있습니다.
#### 순서형 변수의 경우 그 순서대로 수치값을 레이블로 부여하여 간단히 수치화 할 수 있지만,
#### 명목형 변수의 경우 값들의 순서 관계가 없어 수치 레이블링으로는 그 관계를 정확히 표현할 수 없습니다.
#### 그렇기에 명목형 변수의 경우 값들 각각을 새로운 컬럼으로 만들고, 원래 해당하던 값에는 1을,
#### 아닐 경우 0을 부여하는 One-Hot Encoding 방법이 존재합니다. 

In [ ]:
# qualitative column one-hot encoding
qual_col = ['propertyType','suburbName']
ohe = OneHotEncoder(sparse=False)

for i in qual_col:
    train_x = pd.concat([train_x, pd.DataFrame(ohe.fit_transform(train_x[[i]]), columns=ohe.categories_[0])], axis=1)
    
    for qual_value in np.unique(test_x[i]): 
        if qual_value not in np.unique(ohe.categories_): 
            ohe.categories_ = np.append(ohe.categories_, qual_value)
    # One Hot Encoder가 Test 데이터로부터 Fitting되는 것은 Data Leakage이므로, Test 데이터에는 Train 데이터로 Fitting된 One Hot Encoder로부터 transform만 수행되어야 합니다.
    test_x = pd.concat([test_x, pd.DataFrame(ohe.transform(test_x[[i]]), columns=ohe.categories_[0])], axis=1)
    
train_x = train_x.drop(qual_col, axis=1)
test_x = test_x.drop(qual_col, axis=1)
print('Done.')

# Model Hyperparameter Setting

#### 대부분의 모델들은 사람이 직접 설정할 수 있는 Hyperparameter를 가지고 있습니다.
#### 이런 Hyperparameter에 어떤 값이 설정되는가에 따라 모델의 성능은 크게 차이나게 됩니다. 
#### 본 Baseline에서 제공한 Ridge Regression 모델에서는 alpha를 Hyperparameter로 제공했습니다. 
#### alpha는 모델의 규제항으로, 모델의 오버피팅을 방지하는 역할을 합니다.


In [ ]:
Model = Ridge(alpha=1.0) #alpha의 값을 바꿔 규제 정도를 조절할 수 있습니다.

# Model Fit

In [ ]:
Model.fit(train_x, train_y)
print('Done.')

# Prediction

In [ ]:
preds = Model.predict(test_x)
print('Done.')

# Submit

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['monthlyRent(us_dollar)'] = preds
submit.head()

In [ ]:
submit.to_csv('./submit.csv', index=False)